In [1]:
from aicsimageio import AICSImage
import napari
from aicsimageio.readers import CziReader
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Load the image
reader = CziReader("extern_Synlab_2156_17_3_MTB.czi")
# Get whole image
smear = reader.get_image_data("MYX", C=0)

In [3]:
# save in a new variable the information regarding the 673th tile out of 1345
img = smear[673]
img.shape

print(img.shape)
print(np.max(img))
print(np.min(img))

(2048, 1504)
10519
98


In [4]:
class visualization:
    def __init__(self, img):
        self.img = img

    # visualize the image with napari using its numpy array
    def visualize_napari(self):
        """
        :param numpy_img: image to be visualized
        """
        with napari.gui_qt():
            viewer = napari.Viewer()
            viewer.add_image(self.img)

    # visualize different images in the same moment, a list of images is passed as argument and we visualize them all
    def visualize_all_list_napari(self, numpy_img_list: np.ndarray):
        """
        :param numpy_img_list: list containing different images to be visualized
        """
        with napari.gui_qt():
            viewer = napari.Viewer()
            for img in numpy_img_list:
                viewer.add_image(img)
    
    # plot histogram of pixel intensity
    def plot_histogram(self):
        plt.hist(self.img.ravel(), self.img.max(), [0, self.img.max()])
        plt.show()

In [5]:
class thresholding:
    def __init__(self, img):
        self.img = img

    def otsu_thresholding(self):
        """
        Threshold and binarize an image using Otsu's method

        :param image: image you want to threshold
        :return: ret: the computed threshold value
                th: binary image (image with the threshold applied, pixels above threshold are white = 255, pixels below threshold are black= 0)
        """
        ret,th = cv.threshold(self.img, 0, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)
        return ret, th

    def hard_thresholding(self, threshold : int):
        """
        Implement hard threshold (a threshold manually imputed). 
        Take everything above "threshold" to be white and everything below "threshold" to be black.

        :param image: image to be thresholded
        :param threshold: hard threshold to be implemented
        :return: ret: threshold value
                th: binary image (pixels above threshold are white = 255, pixels below threshold are black= 0)
        """
        ret,th = cv.threshold(self.img, threshold, 255, cv.THRESH_BINARY)
        return ret, th

    def adaptive_thresholding(self, block_size : int, c : int):
        """
        Apply adaptive thresholding to the image

        :param image: image to be thresholded
        :param block_size: size of the block used to compute the threshold
        :param c: constant subtracted from the mean or weighted mean
        :return: th: binary image (pixels above threshold are white = 255, pixels below threshold are black= 0)
        """
        th = cv.adaptiveThreshold(self.img, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, block_size, c)
        return th

    def gaussian_thresholding(self, block_size : int, c : int):
        """
        Apply gaussian thresholding to the image

        :param image: image to be thresholded
        :param block_size: size of the block used to compute the threshold
        :param c: constant subtracted from the mean or weighted mean
        :return: th: binary image (pixels above threshold are white = 255, pixels below threshold are black= 0)
        """
        th = cv.adaptiveThreshold(self.img, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, block_size, c)
        return th


In [6]:
class preprocessing: 
    def __init__(self, img):
        self.img = img

    def sharpen(self: np.ndarray):
        """
        :param image: image to be sharpened
        :return: sharp image
        """
        kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
        return cv.filter2D(self, -1, kernel)
    
    # approach in which we sharp (in stead of blur as the example) the image before applying the thresholding
    # sharpen the image using a high-pass filter TODO: can we do this better? sharp out better maybe in sub-images?


In [7]:
#UTILS 

# approach for going through the different tiles and applying the thresholding separately
# split the whole images into tiles
# apply thresholding to each tile
def split_into_tiles(self, tile_size: int):
    """
    split image into tiles of shape tile_size*tile_size

    :param image: image to be split
    :param tile_size: dimensions of single tiles
    :return: tiles: list with the different tiles
    """
    tiles = []
    for i in range(0, self.img.shape[0], tile_size):
        for j in range(0, self.img.shape[1], tile_size):
            tile = self.img[i:i+tile_size, j:j+tile_size]
            tiles.append(tile)
    return tiles


#reconstruct image from different tiles given the number of tiles in x and y direction and a list of tiles
def reconstruct_image(tiles: list, x_tiles: int, y_tiles: int):
    """
    :param tiles:    list with the different single tiles
    :param x_tiles:  how many tiles fit in the x axis
    :param y_tiles:  how many tiles fit in the y axis
    :return:         numpy array, reconstructed image
    """
    big_image = np.zeros((x_tiles*tiles[0].shape[0], y_tiles*tiles[0].shape[1]))
    for i in range(x_tiles):
        for j in range(y_tiles):
            big_image[i*tiles[0].shape[0]:(i+1)*tiles[0].shape[0], j*tiles[0].shape[1]:(j+1)*tiles[0].shape[1]] = tiles[i*y_tiles+j]
    return big_image

In [8]:
class box_creation: 
    def __init__(self, img):
        self.img = img
    

    #connectedComponentsWithStats works better, can get centroid, in this way can put the box/rectangle around the bacilli
    #but not all conncected components are identified, might be a problem of the image we give him
    def get_connected_components_coordinate(self):
        connectivity = 8
        #find connected components
        num_labels, labels_im, stats, centroids = cv.connectedComponentsWithStats(self.img, connectivity)
        #get coordinates of connected components
        coordinates = np.zeros((num_labels, 2),dtype=np.uint64)  #NO uint8

        for i in range(1, num_labels):
            coordinates[i,0] = centroids[i,0]
            coordinates[i,1] = centroids[i,1]
        print(coordinates)
        return coordinates

    # add the 2d bounding boxes to the image
    def add_bounding_boxes(self, coordinates):
        """
        Add whhite rectangles around bacilli

        :param image: image with bacilli to be boxed
        :param coordinates:  coordinates of the center of the bacillus
        """
        for i in range(len(coordinates)):
                x_min = coordinates[i][0]
                x_max = coordinates[i][0]
                y_min = coordinates[i][1]
                y_max = coordinates[i][1]
                cv.rectangle(self.img, (y_min+15, x_min+15), (y_max-15, x_max-15), (255, 0, 0), 4)


    #find connected components with specified size with opencv using connectedComponentsWithStats
    #does not seem to work very good, maybe we do it by hand...
    #could also try with length
    def get_connected_components_with_minimum_and_max_size(self, min_size: int,max_size: int):
        """
        :param img: image where we want to find connected components. black and white immage with
                    int8 rappr.
        :param min_size: minimum size of the cc
        :param max_size: maximum size of the cc
        :return:
        """
        #connecctivity shoud tell us about how connected to they have to be to be considered as one, seems to be working poorly
        connectivity = 8
        #find connected components
        num_labels, labels_im, stats, centroids = cv.connectedComponentsWithStats(self.img, connectivity)
        #get coordinates of connected components
        coordinates =np.array([[0,0]],dtype=np.uint64)
        print(centroids.shape)
        for i in range(1, num_labels):
            if stats[i,4] > min_size and stats[i,4]< max_size:
                coordinates=np.append(coordinates,[centroids[i,:]], axis=0)
        coordinates=coordinates[1:,:]
        return coordinates

    def find_coordinates(image: np.ndarray):  #old version
        """
        Find the coordinates of the connected compenents of the image.
        to be more specific, find first coordinate of a component

        :param image: image where we want to find the connected components
        :return: coordinates: array with coordinates of the components in the rows
        """
        num_conn_comp, labels_conn_comp = cv.connectedComponents(image)
        #num_conn_comp tells us how many components we have
        #labels_conn_comp is an image where the entries where there should be
        # a component are the label of that component
        # overwritten in such a way that the the values of the pixels are the value of the corresponding connected component.
        labels_conn_comp = labels_conn_comp.astype(np.uint64)    #cannot use uint8 cannot rappresent all numbers with uint8

        #save coordinates in two dimensional array
        coordinates = np.zeros((num_conn_comp,2),dtype="int64")

        # iterate over the connected components and add the coordinates of the pixels to the list of coordinates
        for i in range(1, num_conn_comp):
            coordinates[i,0] = np.where(labels_conn_comp == i)[0][0]
            coordinates[i,1] = np.where(labels_conn_comp == i)[1][0]
        return coordinates

In [11]:
class postprocessing: 
    def __init__(self, img):
        self.img = img
    
    #if a black pixel is surrounded by white pixels left right up and down we set it to white, for every pixel in the image
    #to help the connected component function, maybe we can improve this
    def remove_black_pixels_in_white(self):
        """
        :param img: image to be better
        :return:    better immage with less black holes in white parts
        """
        #what is white? 255 or what?
        max=self.img.max()
        #init return image
        ret_img=np.zeros((self.img.shape[0], self.img.shape[1]))

        for i in range(1,self.img.shape[0]-1):
            for j in range(1, self.img.shape[1]-1):
                if self.img[i,j] == 0:
                    if self.img[i-1,j] > 0 or self.img[i+1,j] > 0 or self.img[i,j-1] > 0 or self.img[i,j+1] > 0:
                        ret_img[i,j] = max
                    else:
                        ret_img[i,j]=self.img[i,j]
        return ret_img
    
    #check if image is to be set to 0, if we have more than 400 pixels with value 0 or less then 600 pixels with value 0 we set the image to 0
    def check_image(self):
        """
        For every sub-image we check if its worth keeping or not

        :param img: image to be checked
        :return: bool
        """
        if np.sum(self.img == 0) > 200 and np.sum(self.img == 0) < 800: #maybe check for >0
            return True
        else:
            return False

    #set pixels that are 255 to zero (black)
    def set_zero(img):
        h=img
        h[h>0]=0
        return h

In [10]:
def hard_thresholding_shit_boxes(img: np.ndarray):
    """
    Implement tresholding with a hard treshold (10000).
    Find components and box coordinates, draw the boxes with
    first coordinate of the cc.

    :param img: image to be tresholded
    """
    #sharpen image
    sharpened_img = preprocessing.sharpen(img)
    #hard threshold the sharpened image 10000
    threshold, hard_threshold = thresholding.hard_thresholding(sharpened_img, 10000)
    #convert to uint8 s. t. coonected  component function can accept it
    hard_threshold=np.uint8(hard_threshold)
    #find coordinates of connected components, first coordinate, shit because not centered
    coordinates= box_creation.find_coordinates(hard_threshold)
    #add bounding boxes to the image
    box_creation.add_bounding_boxes(hard_threshold,coordinates)
    #visualize
    visualization.visualize_all_list_napari([hard_threshold,sharpened_img,img])



In [ ]:
def otsu_split_thresholding(img: np.ndarray, tile_size=32):
    """
    Perform Otsu tresholding on sub images of 32 x 32,
    then reconstruct

    :param img:  image to be tresholded
    """
    #sharpen image
    sharpened_img=preprocessing.sharpen(img)
    #split
    tiles=split_into_tiles(sharpened_img,tile_size)
    #otsu on big image
    r_big,th_big=thresholding.otsu_thresholding(sharpened_img)
    #otsu on sub-images
    thresholded_tiles=[]
    for t in tiles:
        r,th=thresholding.otsu_thresholding(t)
        thresholded_tiles.append(th)
    #reconstruct
    reconstructed_image=reconstruct_image(thresholded_tiles,64,47)
    #visualize
    visualization.visualize_all_list_napari([reconstructed_image,sharpened_img,img,th_big])


In [ ]:
def otsu_cleaned_black_split_thresholding(img):
    """
    Performed otsu on an image, then cleaned the image by setting tiles with no basilli to black

    :param img: image to be tresholded
    :return:    tresholded clean image
    """

    sharpened_img= preprocessing.sharpen(img)

    tiles=split_into_tiles(sharpened_img,32)

    thresholded_tiles=[]
    for t in tiles:
        r,th_image=thresholding.otsu_thresholding(t)
        thresholded_tiles.append(th_image)
    reconstructed_image=reconstruct_image(thresholded_tiles,64,47)

    cleaned_tiles=[]
    for tl in thresholded_tiles:
           if postprocessing.check_image(tl):
                m=postprocessing.set_zero(tl)                      #clean in black only thing that changes
                cleaned_tiles.append(m)
           else:
               cleaned_tiles.append(tl)

    reconstructed_clean_image=reconstruct_image(cleaned_tiles,64,47)

    visualization.visualize_all_list_napari([reconstructed_image,reconstructed_clean_image,sharpened_img,img])

    return reconstructed_clean_image


In [ ]:

cleaned_im_black=postprocessing.remove_black_pixels_in_white(cleaned_im_black)
cleaned_im_black_int=np.unit8(cleaned_im_black)

new_coordinates=box_creation.get_connected_components_with_minimum_and_max_size(cleaned_im_black_int, 0,100000)


new_coordinates=new_coordinates.astype(int)

box_creation.add_bounding_boxes(cleaned_im_black,new_coordinates)




images = [img, cleaned_im_black]

visualization.visualize_all_list_napari(images)

